1
、
手写数字识别器
通过
MNIST
数据集训练得到一个手写数字分类器。要求设计一个至
少包含
2
个卷积层和池化层的卷积神经网络。卷积核的尺寸不小于
5*5
，
要求训
后的得到的网络在测试集确率不低于
96%
（
要求在网络中使用
dropout
）
2
、
CIFAR-10
分类网络
通过
CIFAR-10
数据集训练得到一个彩色图像分类网络。要求设计一
个至少包含
5
个卷积层和池化层的卷积神经网络。卷积核的尺寸统一采用
3*3
，
要求训后的得到的网络在测试集上的准确率不低于
70%
（
要求在网络中使用
BatchNorm
）

In [60]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
from torch.utils.data import Dataset,DataLoader, TensorDataset 
from sklearn.utils import shuffle 
import matplotlib. pyplot as plt
data=pd.read_csv("D:\Mis\project\python\dataMine\data\iris.csv")
for i in range(len(data)):
    if data.loc[i,"Species"]=="setosa":
        data.loc[i,"Species"]=0
    if data.loc[i,"Species"]=="versicolor":
        data.loc[i,"Species"]=1
    if data.loc[i,"Species"]=="virginica":
        data.loc[i,"Species"]=2
        
print(data)
data=data.drop("Unnamed: 0", axis=1)
data=shuffle(data) #打散数据 
data.index=range(len(data))


col_titles=["Sepal.Length","Sepal.Width","Petal.Length","Petal.Width"]
for i in col_titles:
    mean,std=data[i].mean(),data[i].std()
    data[i]=(data[i]-mean)/std

# =====数据集处理==================
train_data=data[:-32]
train_x=train_data. drop(["Species"], axis=1).values #删除Species歹U
train_y=train_data["Species"].values.astype(int)
train_x=torch.from_numpy(train_x).type(torch.FloatTensor)
train_y=torch.from_numpy(train_y).type(torch.LongTensor)
test_data=data[-32:]
test_x=test_data. drop(["Species"], axis=1).values #删1 除Species歹!j
test_y=test_data["Species"].values.astype(int)
test_x=torch. from_numpy(test_x).type(torch. FloatTensor)
test_y=torch. from_numpy(test_y).type(torch. LongTensor)
train_dataset=TensorDataset(train_x,train_y)
test_dataset=TensorDataset(test_x,test_y)
train_loader=DataLoader(dataset=train_dataset, batch_size=16, shuffle=True) #训练数据加载器
test_loader=DataLoader(dataset=test_dataset, batch_size=8, shuffle=True)#测试数据加载器
#==========构建网络========== 
class model(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden1=nn.Linear(4, 8)
        self.out=nn.Linear(8, 3)
    def forward(self,x): 
        x=self.hidden1(x) 
        x=F.relu(x) 
        x=self.out(x) 
        return x

    
net=model ()
loss_fn=nn.CrossEntropyLoss()   #交叉燧损失函数
opt=torch.optim.SGD(net.parameters(),lr=0.1) #优化器,学习率设置为0. 01
scheduler = torch.optim.lr_scheduler.MultiStepLR(opt,milestones=[600,700],gamma = 0.5)
#========训练============ 
for epoch in range(1000):
    for i, data in enumerate(train_loader):
        x, y=data
        pred=net(x)
        loss=loss_fn(pred, y)
        
        opt.zero_grad() 
        loss.backward()
        opt.step ()
        scheduler.step()
    if (epoch%10==0): 
        print(loss)
        print(opt.state_dict()['param_groups'][0]['lr'])
def rightness(pred, labels):
    
    pred=torch.max(pred.data,1)[1]
    rights=pred.eq(labels.data.view_as (pred)).sum() 
    return rights, len(labels)
#=========测试(验证)=========
rights=0
length=0
for i, data in enumerate(test_loader):
    x, y=data
    pred=net(x)
    rights=rights+rightness(pred, y)[0] 
    length=length+rightness(pred, y)[1] 
    print(y)
    print(torch.max(pred.data, 1)[1],'\n')
print(rights, length, rights/length)

# ======计算正确率========== 



     Unnamed: 0  Sepal.Length  Sepal.Width  Petal.Length  Petal.Width  Species
0             1           5.1          3.5           1.4          0.2        0
1             2           4.9          3.0           1.4          0.2        0
2             3           4.7          3.2           1.3          0.2        0
3             4           4.6          3.1           1.5          0.2        0
4             5           5.0          3.6           1.4          0.2        0
5             6           5.4          3.9           1.7          0.4        0
6             7           4.6          3.4           1.4          0.3        0
7             8           5.0          3.4           1.5          0.2        0
8             9           4.4          2.9           1.4          0.2        0
9            10           4.9          3.1           1.5          0.1        0
10           11           5.4          3.7           1.5          0.2        0
11           12           4.8          3.4          